# DAT210x - Programming with Python for DS

## Module6- Lab3

This code is intentionally missing! Read the directions on the course lab page!

In [1]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import Normalizer, MaxAbsScaler,MinMaxScaler, KernelCenterer, StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import  Isomap

--INFO : Goals for this assignment are first to see if it's possible to differentiate between people who have **Parkinson's** and who don't using **SciKit-Learn's support vector classifier**, 
and then to take a first-stab at a naive way of fine-tuning your parameters in an attempt to maximize the accuracy of your testing set.



In [2]:
# TODO: Load up the wheat dataset into dataframe 'X'
# Verify you did it properly.
# drop the name column.
# Splice out the status column into a variable y and delete it from X.

In [22]:
X = pd.read_csv('./Datasets/parkinsons.data')
X.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [23]:
X = X.drop('name', axis = 1) 
y = X.status
X = X.drop('status' , axis =1)

In [24]:
#Perform a train/test split. 30% test group size, with a random_state equal to 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 7)

In [6]:
#Create a SVC classifier. 
#Don't specify any parameters, just leave everything as default. 
#Fit it against your training data and then score your testing data
model = SVC()
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print ("Score: ", score)

Score:  0.813559322034


Units on columns: Hz, %, Abs, dB, etc. With all of those units interacting with one another, 
some pre-processing is surely in order.

In [7]:
#SciKit-Learn's pre-processing code. various pre-processors one at a time, 
#checking to see if they improve your predictive accuracy.

#Experiment with Normalizer(), MaxAbsScaler(), MinMaxScaler(), KernelCenterer(), and StandardScaler().
#norm = Normalizer()
#X_train = norm.fit_transform(X_train)
#X_test = norm.transform(X_test)




In [25]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 7)
#max_abs= MaxAbsScaler()
#X_train = max_abs.fit_transform(X_train)
#X_test = max_abs.transform(X_test)


In [13]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 7)
#min_max = MinMaxScaler()
#X_train = min_max.fit_transform(X_train)
#X_test = min_max.transform(X_test)

In [26]:
std_sc = StandardScaler()
X_train = std_sc.fit_transform(X_train)
X_test = std_sc.transform(X_test)

In [16]:
# Immediately after you do the pre-processing, run PCA on your dataset. 
# The original dataset has 22 columns and 1 label column. So try experimenting with PCA n_component values between 4 and 14.
best_score = 0
for n_components in range(4, 15):
    pca = PCA(n_components=n_components)
    pca.fit(X_train)
    X_train2 = pca.transform(X_train)
    X_test2 = pca.transform(X_test)
    for C in np.arange(0.05, 2, 0.05):
        for gamma in np.arange(0.001, 0.1, 0.001):
                model = SVC(C=C, gamma=gamma, kernel='rbf')
                model.fit(X_train2, y_train) 
                score = model.score(X_test2, y_test) 
                if score > best_score:
                    best_score = score
print ('Best score: {}', best_score)   


Best score: {} 0.932203389831


In [8]:
#Program a naive, best-parameter search by creating nested for-loops. 
#The outer for-loop should iterate a variable C from 0.05 to 2, using 0.05 unit increments. 
#The inner for-loop should increment a variable gamma from 0.001 to 0.1, using 0.001 unit increments. 
#As you know, Python ranges won't allow for float intervals, use np.arange
#Since the goal is to find the parameters that result in the model having the best accuracy score, 
#you'll need a best_score = 0 variable that you initialize outside of the for-loops. Inside the inner for-loop, create an SVC model and pass in the C and gamma parameters its class constructor. 
#Train and score the model appropriately. If the current best_score is less than the model's score, update the best_score being sure to print it out, along with the C and gamma values that resulted in it.

In [17]:
gammas = np.arange(0.001, 0.1001, 0.001)
Cs = np.arange(0.05, 2.05, 0.05)
best_score = 0
C_final = 0
gamma_final =0

for C in Cs:
    for gamma in gammas:
        model = SVC(C=C, gamma=gamma)
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        if score > best_score:
            best_score = score
            C_final = C
            gamma_final = gamma


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 7)
svc = SVC(C= C_final, gamma= gamma_final).fit(X_train,y_train)
print('Final C: {}\nFinal gamma: {}\nBest score: {}'.format(svc.C, svc.gamma, best_score))

Final C: 1.5000000000000002
Final gamma: 0.1
Best score: 0.9322033898305084


In [10]:
# Using Normalizer for pre-processing without PCA
#svc = SVC(C= C_final, gamma= gamma_final).fit(X_train,y_train)
#print('Final C: {}\nFinal gamma: {}\nBest score: {}'.format(svc.C, svc.gamma, best_score))


Final C: 0.05
Final gamma: 0.001
Best score: 0.7966101694915254


In [12]:
# Using MaxAbsScaler for pre-processing without PCA
svc = SVC(C= C_final, gamma= gamma_final).fit(X_train,y_train)
print('Final C: {}\nFinal gamma: {}\nBest score: {}'.format(svc.C, svc.gamma, best_score))


Final C: 0.05
Final gamma: 0.001
Best score: 0.7966101694915254


In [14]:
# Using MinMaxScaler for pre-processing without PCA
svc = SVC(C= C_final, gamma= gamma_final).fit(X_train,y_train)
print('Final C: {}\nFinal gamma: {}\nBest score: {}'.format(svc.C, svc.gamma, best_score))

Final C: 0.05
Final gamma: 0.001
Best score: 0.7966101694915254


In [19]:
#In the same spot, run Isomap on the data. 
#Manually experiment with every inclusive combination of n_neighbors between 2 and 5, and n_components between 4 and 6.
#Are you able to get a better accuracy? 

In [27]:
best_score = 0
for n_neighbors in range(2,6):
    for n_components in range(4, 7):
        isomap = Isomap(n_neighbors = n_neighbors, n_components=n_components)
        isomap.fit(X_train)
        X_train2 = isomap.transform(X_train)
        X_test2 = isomap.transform(X_test)
    for C in np.arange(0.05, 2, 0.05):
        for gamma in np.arange(0.001, 0.1, 0.001):
                model = SVC(C=C, gamma=gamma, kernel='rbf')
                model.fit(X_train2, y_train) 
                score = model.score(X_test2, y_test) 
                if score > best_score:
                    best_score = score
print ('Best score: ', best_score)   

Best score:  0.949152542373
